In [33]:

import glob
import os

import sys
import glob
from pathlib import Path, PurePath
path = Path.cwd()
parent_path = path.parents[1]
sys.path.append(str(PurePath(parent_path, 'neuroformer')))
sys.path.append('neuroformer')
sys.path.append('.')
sys.path.append('../')

import pandas as pd
import numpy as np

import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt

from torch.utils.data.dataloader import DataLoader

import math

from neuroformer.model_neuroformer import GPT, GPTConfig
from neuroformer.trainer import Trainer, TrainerConfig
from neuroformer.utils_2 import (set_seed, update_object, 
                                 check_common_attrs, running_jupyter, 
                                 all_device, load_config, update_config, 
                                 dict_to_object, object_to_dict, recursive_print)
from neuroformer.visualize import set_plot_params
from neuroformer.SpikeVidUtils import make_intervals, round_n, SpikeTimeVidData2
import gdown

parent_path = os.path.dirname(os.path.dirname(os.getcwd())) + "/"

import argparse
from neuroformer.SpikeVidUtils import round_n

# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)


In [34]:
running_jupyter()

True

In [43]:
DT = 0.05

import argparse

# Create the parser
parser = argparse.ArgumentParser(description='Specify the dataset')

# Add an argument
parser.add_argument('--dataset', type=str, required=True, help='Dataset to use')

# Parse the arguments
if running_jupyter():
    #namespace
    args = argparse.Namespace(dataset='medial')
else:
    args = parser.parse_args()

# Use the argument
DATASET = args.dataset


In [44]:
""" 

-- DATA --
neuroformer/data/OneCombo3_V1AL/
df = response
video_stack = stimulus
DOWNLOAD DATA URL = https://drive.google.com/drive/folders/1jNvA4f-epdpRmeG9s2E-2Sfo-pwYbjeY?usp=sharing


"""

from neuroformer.prepare_data import DataLinks
from neuroformer.DataUtils import round_n, get_frame_idx
from neuroformer.DataUtils import resample_spikes


if DATASET in ["first", "visnav"]:
    data_path = "./data/VisNav_VR_Expt"
elif DATASET == "medial":
    data_path = "./data/VisNav_VR_Expt/MedialVRDataset/"
elif DATASET == "lateral":
    data_path = "./data/VisNav_VR_Expt/LateralVRDataset"


train_data = pd.read_csv(os.path.join(data_path, "train_data.csv"))

spikes_path = f"{data_path}/NF_1.5/spikerates_dt_0.01.npy"
speed_path = f"{data_path}/NF_1.5/behavior_speed_dt_0.05.npy"
stim_path = f"{data_path}/NF_1.5/stimulus.npy"

spikes = resample_spikes(np.load(spikes_path), 0.01, DT).transpose()
speed = np.round(np.load(speed_path), 3).transpose()
stimulus = np.load(stim_path)

frame_feats = None
print(f"spikes: {spikes.shape}, speed: {speed.shape}")


spikes: (32614, 1906), speed: (32614,)


In [45]:
train_indexes = set([get_frame_idx(value, DT) for value in train_data['Interval']])
test_indexes = set(range(len(speed) - 1)) - train_indexes

print(max(train_indexes), max(test_indexes))

spikes_train = spikes[list(train_indexes)]
print(spikes.shape)
spikes_test = spikes[list(test_indexes)]

speed_train = speed[list(train_indexes)]
speed_test = speed[list(test_indexes)]

print(f"spikes_train: {spikes_train.shape}, spikes_test: {spikes_test.shape}")


293 32612
(32614, 1906)
spikes_train: (293, 1906), spikes_test: (32320, 1906)


In [46]:

max_iterations = np.arange(100, 10000, 900) #default is 5000.
output_dimension = [2, 3, 8, 16, 32] #here, we set as a variable for hypothesis testing below.

print(f"max_iterations: {max_iterations}, output_dimension: {output_dimension}")


max_iterations: [ 100 1000 1900 2800 3700 4600 5500 6400 7300 8200 9100], output_dimension: [2, 3, 8, 16, 32]


In [47]:
import pickle
import matplotlib.pyplot as plt
import cebra
from cebra import CEBRA
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr

max_iterations = np.arange(100, 10000, 1900) #default is 5000.
output_dimension = [2, 3, 8, 16, 32] #here, we set as a variable for hypothesis testing below.
OFFSET = 1

# Store results
results = []



# load results
if DATASET == "medial":
    save_dir = "/share/edc/home/antonis/neuroformer/results/behavior/Medial"
elif DATASET == "lateral":
    save_dir = "/share/edc/home/antonis/neuroformer/results/behavior/Lateral"

with open(os.path.join(save_dir, "cebra_results_1.pkl"), "rb") as f:
    results = pickle.load(f)

In [48]:
results[0].keys()

dict_keys(['max_iterations', 'output_dimension', 'embedding', 'embedding_test', 'prediction', 'pearson_correlation'])

In [49]:
max_corr = (0, 0)
for n, result in enumerate(results):
    corr = result['pearson_correlation']
    if corr > max_corr[0]:
        max_corr = (corr, n)
print(max_corr)

(0.3938572889229721, 0)


In [50]:
max_corr

(0.3938572889229721, 0)